In [1]:
# Part 4

import numpy as np

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1/(1+np.exp(-x))

def sigmoid_prime(x):
    """
    # Derivative of the sigmoid function
    """
    return sigmoid(x) * (1 - sigmoid(x))

learnrate = 0.5
# inputs
x = np.array([1, 2, 3, 4])
# target outputs
y = np.array(0.5)

# Initial weights
w = np.array([0.5, -0.5, 0.3, 0.1])

### Calculate one gradient descent step for each weight
### Note: Some steps have been consolidated, so there are
###       fewer variable names than in the above sample code

# TODO: Calculate the node's linear combination of inputs and weights
# The sum of all inputs*weights
h = np.dot(x,w)

# TODO: Calculate output of neural network
# Result of input*weight is passed to activation function, sigmoid in this case
nn_output = sigmoid(h)

# TODO: Calculate error of neural network
# Error is the difference between the target and the prediction
error = y - nn_output

# TODO: Calculate the error term
#       Remember, this requires the output gradient, which we haven't
#       specifically added a variable for.
error_term = error * sigmoid_prime(h)

# TODO: Calculate change in weights
del_w = learnrate * error_term * x

print('Neural Network output:')
print(nn_output)
print('Amount of Error:')
print(error)
print('Change in Weights:')
print(del_w)

Neural Network output:
0.6899744811276125
Amount of Error:
-0.1899744811276125
Change in Weights:
[-0.02031869 -0.04063738 -0.06095608 -0.08127477]


In [2]:
# Part 5
#%cd "Lesson2_Gradient-Descent"
%run data_prep.py

In [3]:
import numpy as np
from data_prep import features, targets, features_test, targets_test


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output = np.dot(weights, x)

        # cache sigmoid output for later use in sigmoid prime calculation
        yh = sigmoid(output)
        
        # TODO: Calculate the error
        error = y - yh

        # TODO: Calculate the error term
        error_term = error * yh * (1 - yh)

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        # learnrate * derivative of activation function * input values
        del_w += error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.26276093849966364
Train loss:  0.20928619409324895
Train loss:  0.20084292908073417
Train loss:  0.1986215647552789
Train loss:  0.19779851396686018
Train loss:  0.19742577912189863
Train loss:  0.19723507746241067
Train loss:  0.19712945625092465
Train loss:  0.19706766341315077
Train loss:  0.19703005801777368
Prediction accuracy: 0.725


In [4]:
# Part 6
import numpy as np

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1/(1+np.exp(-x))

# Network size
N_input = 4
N_hidden = 3
N_output = 2

np.random.seed(42)
# Make some fake data
X = np.random.randn(4)

weights_input_to_hidden = np.random.normal(0, scale=0.1, size=(N_input, N_hidden))
weights_hidden_to_output = np.random.normal(0, scale=0.1, size=(N_hidden, N_output))


# TODO: Make a forward pass through the network

# [1x4] * [4x3] => [1x3]
hidden_layer_in = np.dot(X, weights_input_to_hidden)
hidden_layer_out = sigmoid(hidden_layer_in)

print('Hidden-layer Output:')
print(hidden_layer_out)

# [1x3] * [3x2] => [1x2]
output_layer_in = np.dot(hidden_layer_out, weights_hidden_to_output)
output_layer_out = sigmoid(output_layer_in)

print('Output-layer Output:')
print(output_layer_out)

Hidden-layer Output:
[0.41492192 0.42604313 0.5002434 ]
Output-layer Output:
[0.49815196 0.48539772]


In [5]:
# Part 7 - Backpropagation

import numpy as np

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


x = np.array([0.5, 0.1, -0.2])
target = 0.6
learnrate = 0.5

weights_input_hidden = np.array([[0.5, -0.6],
                                 [0.1, -0.2],
                                 [0.1, 0.7]])

weights_hidden_output = np.array([0.1, -0.3])

## Forward pass
# [1x3]*[3x2] => [1x2] => hidden layer input
hidden_layer_input = np.dot(x, weights_input_hidden)
# Throw in an activation function
hidden_layer_output = sigmoid(hidden_layer_input)

# [1x2]*[2x1] => [1x1] => output
output_layer_in = np.dot(hidden_layer_output, weights_hidden_output)
# Apply the activation function
output = sigmoid(output_layer_in)

## Backwards pass
## TODO: Calculate output error
error = target - output

# TODO: Calculate error term for output layer
# (y - yp) * sigp(x) == (y-yp) * sig(x) * (1-sig(x)
# [1x1]*[1x1]*(1-[1x1]) => [1x1]
sigp_out = output * (1-output)
print(f"sigp_out shape: {sigp_out.shape}")
output_error_term = error * sigp_out
print(f"output_error_term shape: {output_error_term.shape}\n")

# TODO: Calculate error term for hidden layer
# W * out_error_term * sigp(h) -> hidden layer output
sigp_hidden = hidden_layer_output * (1-hidden_layer_output)
print(f"sigp_hidden shape: {sigp_hidden.shape}")
hidden_error_term = weights_hidden_output * output_error_term * sigp_hidden
print(f"hidden_error_term shape: {hidden_error_term.shape}\n")

# TODO: Calculate change in weights for hidden layer to output layer
# The hidden to output weight step is the 
# learning rate x output unit error x hidden unit activation value.
delta_w_h_o = learnrate * output_error_term * hidden_layer_output
print(f"delta_w_h_o shape: {delta_w_h_o.shape}\n")

# TODO: Calculate change in weights for input layer to hidden layer
# Then, for the input to hidden weights w_iw
# learning rate x hidden unit error x input values.
delta_w_i_h = learnrate * x[:,None] * hidden_error_term
print(f"x shape: {x.shape}")
print(f"x[:,None] shape: {x[:,None].shape}")
print(f"delta_w_h_o shape: {delta_w_i_h.shape}\n")

print('Change in weights for hidden layer to output layer:')
print(delta_w_h_o)
print('Change in weights for input layer to hidden layer:')
print(delta_w_i_h)


sigp_out shape: ()
output_error_term shape: ()

sigp_hidden shape: (2,)
hidden_error_term shape: (2,)

delta_w_h_o shape: (2,)

x shape: (3,)
x[:,None] shape: (3, 1)
delta_w_h_o shape: (3, 2)

Change in weights for hidden layer to output layer:
[0.00804047 0.00555918]
Change in weights for input layer to hidden layer:
[[ 1.77005547e-04 -5.11178506e-04]
 [ 3.54011093e-05 -1.02235701e-04]
 [-7.08022187e-05  2.04471402e-04]]


In [6]:
# Part 8 - Implmenting Backpropagation

import numpy as np
from data_prep import features, targets, features_test, targets_test

np.random.seed(21)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
print (n_records, n_features)

last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))

        ## Backward pass ##
        # TODO: Calculate the network's prediction error
        error = y - output

        # TODO: Calculate error term for the output unit
        # (y - yp) * sigp(x) == (y-yp) * sig(x) * (1-sig(x)
        output_error_term = error * output * (1-output)

        ## propagate errors to hidden layer

        # TODO: Calculate the hidden layer's contribution to the error
        hidden_error = weights_hidden_output * output_error_term
        
        # TODO: Calculate the error term for the hidden layer
        hidden_error_term = hidden_error * hidden_output * (1-hidden_output)
        
        # TODO: Update the change in weights
        del_w_hidden_output +=  output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:,None]

    # TODO: Update weights  (don't forget to division by n_records or number of samples)
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

print(f"hidden_error weights_hidden_output * output_error_term = {weights_hidden_output * output_error_term}")
print(f"hidden_error output_error_term*weights_hidden_output = {output_error_term*weights_hidden_output}")
print(f"hidden_error np.dot(output_error_term, weights_hidden_output) = {np.dot(output_error_term, weights_hidden_output)}")

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))


360 6
Train loss:  0.27630002065852294
Train loss:  0.27487280940102665
Train loss:  0.2734814690053808
Train loss:  0.27212535119812675
Train loss:  0.27080379729958337
Train loss:  0.2695161402601928
Train loss:  0.2682617065761968
Train loss:  0.26703981808591765
Train loss:  0.2658497936485804
Train loss:  0.26469095070807397
hidden_error weights_hidden_output * output_error_term = [-0.07199042  0.05513622]
hidden_error output_error_term*weights_hidden_output = [-0.07199042  0.05513622]
hidden_error np.dot(output_error_term, weights_hidden_output) = [-0.07199042  0.05513622]
Prediction accuracy: 0.425
